In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer

import numpy as np
import itertools

In [4]:
topic2 = """
Topic 2: Introduction to Pharmacodynamics1Pharmacology – An overview2Drugs act on cellsCytoplasm : encloses the contents of the cellNucleus :acts as the 'control centre' of the cell.-contains the genetic code, DNA and blueprintsfor construction of all the cell's proteins .Boundary wall called the cell membrane :made up of phospholipids and proteins.3Biological targets of drugs Receptors EnzymesProteinsDrug4ReceptorsReceptors are proteins usually on the cell membrane that are involved in communication and coordination between and within cells
Video5EnzymesEnzymes are proteins that catalyze chemical reactions in the cell
6Active SiteWhere drug bindsVideo – changes in protein conformation7Drug Receptor TheoryMolecules that interact with receptors are called ligands.Each receptor commonly has at least 1 naturally occurringligand that interacts with its binding site
This is called theligand binding site
When a ligand binds, it switches on the receptor and a message is propagated.8it changes the shape of the receptorWhen the ligand binds,protein once it has binds into the binding site
It does not fit thebinding site exactly
Instead, as it binds at the site, various attractionsbetween the ligand the functional groups on the binding site pulltoward each other, distorting the shape of the rest of the protein.Effect of a ligand-receptor interaction:• Activation of a membrane bound enzyme.9• Release of secondary messengers within the cell’s membrane which migrate to other parts of the membrane and either activate or inhibit further enzymesVideoG-Protein10• Open (or close) an ion channel through the membrane, allowing hydrophilic ions to pass through the hydrophobic cell membrane.VideoVideo211Agonist vs Antagonist Agonists and antagonists are ligands of a receptor.  Most drugs that you see in the market are either agonists or antagonists of either receptors, or enzymes.Natural ligand = endogenous ligand12Agonist Agonists often have structures that are similar to that of the endogenous ligand.  Endogenous ligands are naturally occurring molecules in the body that have innate biological functions.  An agonist is a molecule that gives rise to biological effect
It mimics the endogenous ligand.  Both endogenous ligand and agonists produces similar biological effect.  When the biological response is lacking in a disease, an agonist drug can restore the biological function.13Antagonist Binds to the receptor but does not produce a biological effect.  Prevents the binding of the endogenous ligand or agonist. Blocks the action of a receptor
This would reduce the effect of an endogenous ligand.  When the biological response is too overwhelming, a drug that serves as an antagonist would restore the biological function.14Difference between Biological Effect and Biological Response15Competitive vs Non-competitiveAntagonists / InhibitorsCompetitiveNon-competitive16Summary of Agonist & Antagonists17SummaryCompetitiveNon-competitiveAgonistAntagonist• Activation of Membrane-bound enzyme• Release of Secondary Messengers• Open/Close Ion-channels18• Activation of membrane Antagonistbound enzyme• Endogenous ligand• Non-competitive 19• Opening Ion Channel• Endogenous ligand• Competitive Antagonist20• Release of secondary messengers• Agonist• Competitive Antagonist21Pharmacodynamics of drug bindingThe binding of a drug molecule to the active site of a receptor is classified into 2 types: Irreversible binding and reversible binding.Irreversible binding Involves strong covalent bondsDrug molecule cannot detach from the receptor active siteDrug + Receptor (Drug – Receptor) Complex22Reversible bindingInvolves ionic interaction, hydrogen bonds, ion dipole interactions, van der waals interactions and permanent dipole interacts.Drug molecule can be unbound from the active site because drug binding and unbinding is in an equilibrium process.Drug + Receptor (Drug – Receptor) ComplexLe Chatelier’s principle?Biological EffectWhy is it much more favourable to have a drug that binds reversibly at the receptor?Why do potent drugs usually require a lower dose and weaker drug require a higher dose?23Why do potent drugs usually require a lower dose and weaker drug require a higher dose?Strong Drug ADrug + Receptor(Drug – Receptor) ComplexDrug + Receptor(Drug – Receptor) ComplexWeak Drug B24 The strength of the interaction between a drug molecule and its receptor can be determined the equilibrium constant , Kd for the interaction:Kd reflects how strong the drug molecule have bound.Drug + Receptor (Drug – Receptor) ComplexKd(Dissociation reaction – backward reaction)25Dose Response CurveCopy the graph on to your lecture notes26Dose Response Curve EC50 = The plasma drug concentration required to achieve 50% of the maximum biological response. ED50 = The effective dose required to achieve 50% of the maximum biological response. Question 1: If I have 2 similar agonists “A” and “B” and the dose response curve of drug A is provided in figure 2.12, how would the graph of B look like if B is a more potent drug? [Assuming that both give the same maximum biological response]27The lower the EC50 value, the stronger the agonist28 Question 2: How would a dose response curve of an antagonist drug look like?The lower the IC50 value, the stronger the antagonist29 IC50 (half maximal inhibitory concentration) is the concentration of the drug required to achieve 50% inhibition
This applies for competitive enzyme inhibitors and competitive antagonists.  Question 3: If I have 2 similar antagonists “A” and “B”, and B is a stronger antagonist than A, how would the dose response curve look like? Indicate in your graph above.30SummaryChemistryAgonist vs AntagonistCompetitive vs Non-competitive•••Activation of Membrane-bound enzymeRelease of Secondary MessengersOpen/Close Ion-channelsDrug + Receptor(Drug – Receptor) Complex(Drug Topics)Intermolecular Interactions(Next Topic)∑ (Signals)Biological EffectBiological ResponseBiology31

      """

microcontrollers = """
1.1 What is a micro controller?  
 
A microcontroller MCU or  C is a small computer on a single integrated circuit IC 
consisting internally of a relatively simple CPU, clock, timers, IO ports and memory 
program memory in the form of flash, and a small amount of data memory in the form of 
RAM. Microcontrollers are designed for small or dedicated applications.  
 
Thus, in contrast to the micro-processor used in personal computers PC and other 
high-performance or general purpose applications , simplicity is emphasized in 
microcontrollers.  
 
Some microcontrollers may use four -bit words and operate at clock rate as low as 4 kHz, 
as this is adequate for many typical applications, enabling low power current 
consumption in the micro Ampere range. They will generally have the ability to retain 
functionality while waiting for an event such as a button press or other interrupt power 
consumption while sleeping CPU clock and most peripherals off may just be nanowatts, 
making many of them well suited for long lasting battery applications.  
 
Other microcontrollers may serve performance-critical roles, where they may need to act 
more like a digital signal processor DSP, with higher clock speeds and power 
consumption. Microcontrollers are used in automatic ally controlled products and devices, 
such as automobile engine control systems, implantable medical devices, remote controls, office machines, appliances, power tools and toys.  
 
By reducing the size and cost compared to a design that uses a separate micr o-
processor, memory, and inputoutput devices, microcontrollers make it economical to 
digitally control even more devices and processes. 
 
Mixed signal microcontrollers are common, integrating analogue components needed to 
control non- digital electronic sys tems. 

"""

In [49]:
n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([microcontrollers])
candidates = count.get_feature_names_out()

In [50]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
doc_embedding = model.encode([microcontrollers])
candidate_embeddings = model.encode(candidates)

In [51]:
def max_sum_sim(doc_embedding, word_embeddings, words, top_n, nr_candidates):
    # Calculate distances and extract keywords
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # Get top_n words as candidates based on cosine similarity
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # Calculate the combination of words that are the least similar to each other
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

max_sum_sim(doc_embedding, candidate_embeddings,candidates, top_n=10, nr_candidates=20)

['battery',
 'economical',
 'digitally',
 'toys',
 'tools',
 'small',
 'cpu',
 'computer',
 'pc',
 'nanowatts']

In [6]:
kw_model = KeyBERT()
kw_model.extract_keywords(docs=[topic2], vectorizer=KeyphraseCountVectorizer())

[('protein conformation7drug receptor theorymolecules', 0.6111),
 ('drug bindingthe binding', 0.5496),
 ('drug molecule', 0.52),
 ('6active sitewhere drug bindsvideo', 0.506),
 ('receptors', 0.4974)]